In [4]:
import os
import requests
import zipfile
import pandas as pd

In [5]:
url = 'https://github.com/ProfLeandroLessa/classroom-datasets/raw/master/CDD/TP/pesquisa_preferencias.zip'
 
response = requests.get(url)

In [6]:
diretorio_destino = 'C:\\survey_data'

if response.status_code == requests.codes.OK:
    nome_arquivo_zip = url.split('/')[-1]
    caminho_arquivo_zip = os.path.join(diretorio_destino, nome_arquivo_zip)

In [7]:
with open(caminho_arquivo_zip,'wb') as arquivo:
    arquivo.write(response.content)
    print("Download finalizado. Salvo em {}".format(caminho_arquivo_zip))

Download finalizado. Salvo em C:\survey_data\pesquisa_preferencias.zip


In [8]:
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
    zip_ref.extractall(diretorio_destino)

print(f"Arquivo ZIP descompactado em: {diretorio_destino}")

Arquivo ZIP descompactado em: C:\survey_data


In [9]:
#CRIAR UM ARQUIVO COM TODOS OS ARQUIVOS DA PASTA
lista_arquivo = [] #cria uma lista em branco
lista_path = os.listdir(diretorio_destino) #salva os nomes dos arquivos dentro da pasta

In [12]:
print(len(lista_path))

364


In [13]:
for arquivo in lista_path: #varre os arquivos na pasta e junta em um unico dataframe
    local_arquivo = os.path.join(diretorio_destino, arquivo)
    if os.path.isfile(local_arquivo) and local_arquivo.endswith('.csv'): #verifica se o arquivo é um csv válido
        try:
            df = pd.read_csv(local_arquivo, sep = '|', encoding = 'latin-1')
            lista_arquivo.append(df)
        except Exception as e:
            print(f"Erro ao ler arquivo {local_arquivo}:{e}")
    else:
        print(f"{local_arquivo} não é um arquivo CSV válido ou não existe")

C:\survey_data\pesquisa_preferencias.zip não é um arquivo CSV válido ou não existe


In [14]:
print(len(lista_arquivo))

363


In [15]:
df.head(5)

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10142416,2020-12-31,Masculino,1996-05-07,gato,frio,Cerveja,Pintar quadros
1,10142475,2020-12-31,Masculino,1955-05-30,peixe,quente,Café,Praticar esporte
2,10142522,2020-12-31,Masculino,1954-12-22,peixe,moderado,Cerveja,Praticar esporte
3,10142881,2020-12-31,Masculino,1952-07-30,gato,frio,refrigerante,Assistir TV
4,10143184,2020-12-31,Masculino,1984-06-05,tartaruga,quente,refrigerante,Pintar quadros


In [18]:
df_total = pd.concat(lista_arquivo)

In [20]:
df_total.count()

cod_pessoa          2127
data_coleta         2127
genero              2127
data_nascimento     2127
animal_estimacao    2127
clima               2127
bebida_favorita     2127
hobbies             2127
dtype: int64

In [21]:
#transformar o arquivo em um unico arquivo csv

df_total.to_csv(diretorio_destino +'\pesquisas_completo.csv', sep = ";", encoding = 'latin-1', index = False)

In [22]:
arquivo = os.path.join(diretorio_destino,'pesquisas_completo.csv')

pesquisa_completo = pd.read_csv(arquivo, sep = ';', encoding= 'latin-1')

In [23]:
pesquisa_completo.count()

cod_pessoa          2127
data_coleta         2127
genero              2127
data_nascimento     2127
animal_estimacao    2127
clima               2127
bebida_favorita     2127
hobbies             2127
dtype: int64

In [ ]:
#verifica valores nulos
pesquisa_completo.isna().sum()

In [ ]:
#bibliotecas para conectar com o banco de dados MySQL

import MySQLdb
import sqlalchemy
import mysql.connector

In [ ]:
#conexão com a base de dados

user = 'root'
password = 'IGTExpKAKA'
host = 'localhost'
port = '3307'
database = 'pesquisa_db'

string_conexao = f'mysql://{user}:{password}@{host}:{port}/{database}'
string_conexao

In [ ]:
engine = sqlalchemy.create_engine(string_conexao, future=True)

In [ ]:
conn = engine.connect()

In [ ]:
animal = list(pesquisa_completo.animal_estimacao.unique())
print(animal)

In [ ]:
for valor in animal:
    try:
        query = f"""INSERT INTO animal_estimacao (animal) VALUES ('{valor}')"""
        conn.execute(sqlalchemy.text(query))
        conn.commit()
        print(f"Registro inserido com sucesso: {valor}")
    except Exception as e:
        print(f"Não foi possivel inserir o registro {valor}. O erro encontrado foi {e}")

In [ ]:
bebida = list(pesquisa_completo.bebida_favorita.unique())
print(bebida)

In [ ]:
#insere o valor na tabela verificando se tem duplicadas com base na coluna bebida

for valor in bebida:
    try:
        query_select = f"""SELECT COUNT(*) FROM bebida WHERE bebida = '{valor}'"""
        result = conn.execute(sqlalchemy.text(query_select)).scalar()
        if result == 0:
            query = f"""INSERT INTO bebida (bebida) VALUES ('{valor}')"""
            conn.execute(sqlalchemy.text(query))
            conn.commit()
            print(f"Registro inserido com sucesso {valor}")
        else:
            print(f"Registro já existe no banco de dados: {valor}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {valor}. Erro: {e}.")

In [ ]:
clima = list(pesquisa_completo.clima.unique())
print(clima)

In [ ]:
for valor in clima:
    try:
        query_select = f"""SELECT COUNT(*) FROM clima WHERE clima = '{valor}'"""
        result = conn.execute(sqlalchemy.text(query_select)).scalar()
        if result == 0:
            query = f"""INSERT INTO clima (clima) VALUES ('{valor}')"""
            conn.execute(sqlalchemy.text(query))
            conn.commit()
            print(f"Registro inserido com sucesso: {valor}")
        else:
            print(f"Registro já existe no banco de dados: {valor}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {valor}. Erro: {e}.")

In [ ]:
hobbie = list(pesquisa_completo.hobbies.unique())
print(hobbie)

In [ ]:
for valor in hobbie:
    try:
        query_select = f"""SELECT COUNT(*) FROM hobbie WHERE hobbie = '{valor}'"""
        result = conn.execute(sqlalchemy.text(query_select)).scalar()
        if result == 0:
            query = f"""INSERT INTO hobbie (hobbie) VALUES ('{valor}')"""
            conn.execute(sqlalchemy.text(query))
            conn.commit()
            print(f"Registro inserido com sucesso: {valor}")
        else:
            print(f"Registro já existe no banco de dados: {valor}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {valor}. Erro: {e}.")

In [ ]:
df_pessoas = pesquisa_completo[['cod_pessoa','genero','data_nascimento']]

In [ ]:
df_pessoas.head(5)

In [ ]:
for valor in df_pessoas.itertuples():
    cod_pessoa = valor.cod_pessoa
    genero = valor.genero
    data_nascimento = valor.data_nascimento
    try:
        query_select = f"""SELECT COUNT(*) FROM pessoa WHERE cod_pessoa = '{cod_pessoa}'"""
        result = conn.execute(sqlalchemy.text(query_select)).scalar()
        if result == 0:
            query = f"""INSERT INTO pessoa (cod_pessoa, genero, data_nascimento)
                        VALUES ('{cod_pessoa}','{genero}','{data_nascimento}')"""
            conn.execute(sqlalchemy.text(query))
            conn.commit()
            print(f"Registro atualizado: {cod_pessoa}")
        else:
            print(f"Registro já existe: {cod_pessoa}")
    except Exception as e:
        print(f"Registro não atualizado. Erro: {e}")

In [ ]:
#validando se foram apenas valores unicos para a base de dados
valors = pesquisa_completo['cod_pessoa'].nunique()
print(valors)

In [ ]:
df_tb_animal_ = pd.read_sql('animal_estimacao', con = conn)
dict_tb_animal = df_tb_animal_.set_index(df_tb_animal_.animal)['cod_animal_estimacao'].to_dict()
dict_tb_animal

In [ ]:
pesquisa_completo.animal_estimacao = pesquisa_completo.animal_estimacao.map(dict_tb_animal)
pesquisa_completo.head(5)

In [ ]:
df_tb_bebida = pd.read_sql('bebida', con = conn)
dict_tb_bebida = df_tb_bebida.set_index(df_tb_bebida.bebida)['cod_bebida'].to_dict()
dict_tb_bebida

In [ ]:
pesquisa_completo.bebida_favorita = pesquisa_completo.bebida_favorita.map(dict_tb_bebida)
pesquisa_completo.head(5)

In [ ]:
df_tb_hobbie = pd.read_sql('hobbie', con = conn)
dict_tb_hobbie = df_tb_hobbie.set_index(df_tb_hobbie.hobbie)['cod_hobbie'].to_dict()
dict_tb_hobbie

In [ ]:
pesquisa_completo.hobbies = pesquisa_completo.hobbies.map(dict_tb_hobbie)
pesquisa_completo.head(5)

In [ ]:
df_tb_clima = pd.read_sql('clima', con = conn)
dict_tb_clima = df_tb_clima.set_index(df_tb_clima.clima)['cod_clima'].to_dict()
dict_tb_clima

In [ ]:
pesquisa_completo.clima = pesquisa_completo.clima.map(dict_tb_clima)
pesquisa_completo.head(5)

In [ ]:
for valor in pesquisa_completo.itertuples():
    data_pesquisa = valor.data_coleta
    cod_pessoa = valor.cod_pessoa
    cod_animal_estimacao = valor.animal_estimacao
    cod_bebida = valor.bebida_favorita
    cod_hobbie = valor.hobbies
    cod_clima = valor.clima
    try:
        query_select = f"""SELECT COUNT(*) FROM pesquisa WHERE cod_pessoa = '{cod_pessoa}'"""
        result = conn.execute(sqlalchemy.text(query_select)).scalar()
        if result == 0:
            query = f"""INSERT INTO pesquisa (data_pesquisa, cod_pessoa, cod_animal_estimacao, cod_bebida, cod_hobbie, cod_clima)
                        VALUES ('{data_pesquisa}','{cod_pessoa}','{cod_animal_estimacao}','{cod_bebida}','{cod_hobbie}','{cod_clima}')"""
            conn.execute(sqlalchemy.text(query))
            conn.commit()
            print(f"Registro inserido: {cod_pessoa}")
        else:
            print(f"Registro já existe: {cod_pessoa}")
    except Exception as e:
        print(f"Registro não atualizado. Erro: {e}.")
        